# Détection d'anomalies dans les logs système

## Introduction et justification du modèle

Ce notebook implémente un système de détection d'anomalies dans les logs système en utilisant l'algorithme Isolation Forest. Le choix de cet algorithme est justifié par plusieurs raisons :

1. **Non-supervisé** : Isolation Forest ne nécessite pas de données étiquetées, ce qui est idéal pour la détection d'anomalies dans les logs où les exemples d'anomalies sont rares.

2. **Scalabilité** : L'algorithme a une complexité temporelle linéaire et peut gérer de grands volumes de données efficacement grâce à son approche par sous-échantillonnage.

3. **Robustesse aux données haute dimension** : Après le one-hot encoding, nos données ont une très haute dimensionnalité. Isolation Forest reste efficace dans ce contexte.

4. **Efficacité** : L'algorithme est particulièrement efficace pour détecter les anomalies car il se base sur le principe que les anomalies sont plus faciles à isoler que les observations normales.

In [ ]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score
import joblib
from datetime import datetime
import json

## Analyse exploratoire des données

Avant de procéder au traitement des données et à l'entraînement du modèle, il est important d'analyser nos données pour mieux comprendre leur structure et leurs caractéristiques.

In [ ]:
def analyze_data(df):
    print("Analyse exploratoire des données:")
    print("\nDimensionalité des données:")
    print(f"Nombre de lignes: {df.shape[0]}")
    print(f"Nombre de colonnes: {df.shape[1]}")
    
    print("\nDistribution des processus:")
    process_dist = df['Process'].value_counts()
    print(process_dist.head())
    
    print("\nDistribution des hosts:")
    host_dist = df['Hostname'].value_counts()
    print(host_dist.head())
    
    # Visualisation de la distribution des processus
    plt.figure(figsize=(12, 6))
    process_dist.head(10).plot(kind='bar')
    plt.title('Top 10 des processus les plus fréquents')
    plt.xlabel('Processus')
    plt.ylabel('Nombre d\'occurrences')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Chargement et analyse des données
print("Chargement du fichier CSV...")
df = pd.read_csv("all_logs.csv")
analyze_data(df)

## Prétraitement des données

Cette section comprend le nettoyage des données et l'encodage one-hot des variables catégorielles.

In [ ]:
# Nettoyage des données
print("Vérification des valeurs nulles avant suppression :")
print(df.isnull().sum())

print("\nSuppression des lignes avec des valeurs nulles...")
df_cleaned = df.dropna()

print(f"\nNombre de lignes après suppression des valeurs nulles : {len(df_cleaned)}")

# Conversion des types
df_cleaned[["Date", "Hostname", "Process", "IdProcess", "Message"]] = \
    df_cleaned[["Date", "Hostname", "Process", "IdProcess", "Message"]].astype(str)

# One-hot encoding
print("\nApplication du One-Hot Encoding...")
df_encoded = pd.get_dummies(df_cleaned, columns=["Date", "Hostname", "Process", "IdProcess", "Message"])
print(f"Le DataFrame encodé contient désormais {df_encoded.shape[1]} colonnes.")

## Fonctions d'évaluation du modèle

Ces fonctions permettent d'évaluer les performances du modèle et d'analyser les anomalies détectées.

In [ ]:
def evaluate_model(model, X_test, contamination=0.1):
    # Prédictions sur le jeu de test
    predictions = model.predict(X_test)
    
    # Convertir les prédictions (-1: anomalie, 1: normal) en format binaire
    predictions_binary = np.where(predictions == -1, 1, 0)
    
    # Créer un "ground truth" synthétique basé sur les scores d'anomalie
    scores = -model.score_samples(X_test)
    threshold = np.percentile(scores, (1 - contamination) * 100)
    ground_truth = np.where(scores > threshold, 1, 0)
    
    # Calculer les métriques
    precision = precision_score(ground_truth, predictions_binary)
    recall = recall_score(ground_truth, predictions_binary)
    f1 = f1_score(ground_truth, predictions_binary)
    
    print("\nÉvaluation du modèle:")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1-score: {f1:.3f}")
    
    # Visualisation des scores d'anomalie
    plt.figure(figsize=(10, 6))
    plt.hist(scores, bins=50)
    plt.axvline(x=threshold, color='r', linestyle='--', label='Seuil d\'anomalie')
    plt.title('Distribution des scores d\'anomalie')
    plt.xlabel('Score d\'anomalie')
    plt.ylabel('Nombre d\'observations')
    plt.legend()
    plt.show()
    
    return predictions_binary, scores

def analyze_anomalies(df_original, predictions, scores):
    df_anomalies = df_original.copy()
    df_anomalies['anomaly_score'] = scores
    df_anomalies['is_anomaly'] = predictions
    
    print("\nAnalyse des anomalies détectées:")
    print(f"Nombre total d'anomalies: {predictions.sum()}")
    print("\nExemples d'anomalies détectées (top 5 par score):")
    anomalies = df_anomalies[df_anomalies['is_anomaly'] == 1].sort_values('anomaly_score', ascending=False)
    print(anomalies[['Date', 'Hostname', 'Process', 'Message', 'anomaly_score']].head())
    
    # Visualisation des anomalies par host
    plt.figure(figsize=(10, 6))
    anomalies['Hostname'].value_counts().head(10).plot(kind='bar')
    plt.title('Distribution des anomalies par host')
    plt.xlabel('Hostname')
    plt.ylabel('Nombre d\'anomalies')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## Entraînement du modèle

Cette section comprend la séparation des données, l'entraînement du modèle et son évaluation.

In [ ]:
# Séparation des données
print("Séparation des données...")
X_train, X_test = train_test_split(df_encoded, test_size=0.2, random_state=42)

# Configuration et entraînement du modèle
print("\nEntraînement du modèle...")
model = IsolationForest(
    n_estimators=100,
    contamination='auto',
    random_state=42
)
model.fit(X_train)

# Évaluation du modèle
predictions, scores = evaluate_model(model, X_test)
analyze_anomalies(df_cleaned.loc[X_test.index], predictions, scores)

## Sauvegarde du modèle

Sauvegarde du modèle entraîné et de ses métadonnées pour une utilisation future.

In [ ]:
# Sauvegarde du modèle et des métadonnées
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f'isolation_forest_model_{timestamp}.joblib'
metadata_filename = f'model_metadata_{timestamp}.json'

# Création des métadonnées
metadata = {
    'date_entrainement': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'nombre_lignes': len(df),
    'nombre_colonnes_encodees': df_encoded.shape[1],
    'parametres_model': {
        'n_estimators': 100,
        'contamination': 'auto'
    }
}

# Sauvegarde
joblib.dump(model, model_filename)
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f)

print(f"\nModèle sauvegardé: {model_filename}")
print(f"Métadonnées sauvegardées: {metadata_filename}")

## Conclusion

Ce notebook présente une implémentation complète d'un système de détection d'anomalies dans les logs système utilisant l'algorithme Isolation Forest. Les principales améliorations apportées sont :

1. Une analyse exploratoire approfondie des données
2. Une évaluation rigoureuse du modèle avec des métriques performance (précision, rappel, F1-score)
3. Une visualisation détaillée des résultats et des anomalies détectées
4. Un système de sauvegarde du modèle avec métadonnées

### Limitations et améliorations possibles

1. **Prétraitement des données**:
   - Le one-hot encoding génère un grand nombre de features (19578 colonnes)
   - Une réduction de dimensionnalité (PCA, UMAP) pourrait être envisagée
   - L'utilisation de techniques de feature engineering plus avancées pour les logs (ex: TF-IDF pour les messages)

2. **Modélisation**:
   - Tester d'autres algorithmes de détection d'anomalies (DBSCAN, LOF, auto-encoders)
   - Optimiser les hyperparamètres via validation croisée
   - Implémenter une approche ensembliste combinant plusieurs modèles

3. **Évaluation**:
   - Créer un jeu de données étiqueté pour une validation plus rigoureuse
   - Implémenter une évaluation en temps réel
   - Ajouter des métriques spécifiques au domaine des logs

4. **Production**:
   - Ajouter un système de monitoring des performances du modèle
   - Implémenter une stratégie de réentraînement périodique
   - Optimiser le pipeline pour le traitement en temps réel

### Perspectives

Ce système pourrait être étendu pour :
1. Détecter des patterns d'anomalies spécifiques
2. Intégrer des règles métier pour la classification des anomalies
3. Générer des alertes automatiques avec différents niveaux de criticité
4. S'adapter automatiquement aux changements de patterns dans les logs

Le code source est disponible pour toute amélioration ou adaptation à des besoins spécifiques.